In [1]:
import pandas as pd
import os
from haversine import haversine
import folium
os.chdir('C:/Users/user/Desktop/서울시청/데이터')

* 최소 거리 계산

In [2]:
def dis_cal(grid,b):
    dis_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        dis_min=999999999999999999
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<dis_min:
                dis_min=dis
        dis_l.append(dis_min)
        if i%40000==0:
            print(i)
    return dis_l

* 반경 100m count

In [3]:
def count_100cal(grid,b):
    count_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        count=0
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<=100:
                count+=1
        count_l.append(count)
        if i%40000==0:
            print(i)
    return count_l

* 반경 100m 존재 여부

In [4]:
def is_100cal(grid,b):
    count_l=[]
    A_data=list(zip(grid.위도,grid.경도))
    B_data=list(zip(b.위도,b.경도))
    length_A=len(A_data)
    length_B=len(B_data)
    for i in range(length_A):
        count=0
        for j in range(length_B):
            dis=haversine(A_data[i],B_data[j],unit='m')
            if dis<=100:
                count=1
                count_l.append(count)
                break
        if count==0:
            count_l.append(count)
        if i%40000==0:
            print(i)
    return count_l

### 중,고등학교 전처리

In [6]:
school=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/학교/중고등학교위치_수정.csv',encoding='cp949',index_col=0)
school.columns

Index(['학교ID', '학교명', '학교급구분', '설립일자', '설립형태', '본교분교구분', '운영상태', '소재지지번주소',
       '소재지도로명주소', '시도교육청코드', '시도교육청명', '교육지원청코드', '교육지원청명', '생성일자', '변경일자',
       '위도', '경도', '데이터기준일자'],
      dtype='object')

In [7]:
school=school[['학교명','위도','경도']]

In [8]:
school.reset_index(drop=True,inplace=True)

In [9]:
school.head()

,학교명,위도,경도
0,서울문영여자고등학교,37.472905,126.950918
1,봉은중학교,37.518977,127.062389
2,선유중학교,37.533286,126.894329
3,독산고등학교,37.476219,126.910973
4,대영고등학교,37.503661,126.905866


### 문화, 편의시설 데이터 불러오기(전처리 및 병합)

In [10]:
sports=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/스포츠센터병합.csv',encoding='cp949',index_col=0)
adult=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/유흥업소.csv',encoding='cp949',index_col=0)
study=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/학업시설.csv',encoding='cp949',index_col=0)
market=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/편의점,마켓.csv',encoding='cp949',index_col=0)
game=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/오락시설.csv',encoding='cp949',index_col=0)
house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/숙박 시설.csv',encoding='cp949',index_col=0)
culture=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/문화,쇼핑.csv',encoding='cp949',index_col=0)
hell=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/지옥.csv',encoding='cp949',index_col=0)
parking=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/주차장병합.csv',encoding='cp949',index_col=0)

### 체육시설 빅데이터 캠퍼스 데이터 병합
* QGIS 좌표계 위도, 경도(4326) 변환 후 기존 데이터와 병합

In [ ]:
gong_sports=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/441. 서울시 공공체육시설 공간데이터/2. 파일데이터/2017/공공체육시설/TL_PUB_ALSFC/공공체육시설_4326.csv',encoding='cp949')
sports=sports[['상호명','위도','경도']]

In [ ]:
gong_sports=gong_sports[['ALSFC_NM','y','x']]
gong_sports.columns=sports.columns

In [ ]:
sports=pd.concat([sports,gong_sports])
sports.reset_index(drop=True,inplace=True)
sports.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/스포츠센터병합.csv',encoding='cp949')

## 자전거 편의시설 전처리
* QGIS 좌표계 위도, 경도(4326) 변환 후 처리

In [ ]:
bike=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/408. 서울시 자전거 편의시설 공간데이터/2. 파일데이터/2017/자전거 인프라 현황/TL_BCYCL_INFR_ST_2017/자전거인프라_4326.csv',encoding='cp949')
bike=bike[(bike.TGRP_CODE==6)|(bike.TGRP_CODE==8)|(bike.TGRP_CODE==9)|(bike.TGRP_CODE==10)|(bike.TGRP_CODE==11)|(bike.TGRP_CODE==12)|(bike.TGRP_CODE==14)]
bike.reset_index(drop=True,inplace=True)
bike.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/자전거 편의시설.csv',encoding='cp949')

In [16]:
bike=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/자전거 편의시설.csv',encoding='cp949',index_col=0)

### 주차장 빅데이터 캠퍼스 데이터 병합
* QGIS 좌표계 위도, 경도(4326) 변환 후 기존 데이터와 병합

In [ ]:
parking_outdoor=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공간데이터 병합/410. 서울시 노외주차장 공간데이터/2. 파일데이터/2017/주차장 정보(노외주차장)/TL_PRKPLCE_ST_2017/노외주차장.csv',encoding='cp949')

In [ ]:
parking=parking[['상호명','위도','경도']]
parking_outdoor=parking_outdoor[['BD_15_NM','위도','경도']]
parking_outdoor.columns=parking.columns
parking=pd.concat([parking,parking_outdoor])
parking.reset_index(drop=True,inplace=True)
parking.to_csv('C:/Users/user/Desktop/서울시청/데이터/버스정류장관련/주차장병합.csv',encoding='cp949')

### 자전거 인프라 전처리
* QGIS를 활용하여 인프라를 포함하는 격자 추출

In [17]:
bike

,ID,TGRP_CODE,ADDR_KOR,위도,경도
0,174,12,서울특별시 강동구 천호동 484-9,37.543813,127.115989
1,175,12,서울특별시 강동구 천호동 485-12,37.543935,127.116303
2,176,12,서울특별시 마포구 마포동 425-1,37.536410,126.941293
3,177,12,서울특별시 광진구 광장동 557-1,37.543949,127.108888
4,178,12,서울특별시 용산구 이촌동 359,37.511094,126.981893
...,...,...,...,...,...
1110,3143,14,서울특별시 강남구 테헤란로 201,37.501409,127.036784
1111,3144,14,서울특별시 강남구 논현동 1,37.515812,127.019980
1112,3145,14,서울특별시 강남구 논현로 867,37.526079,127.028501
1113,3146,14,서울특별시 강남구 학동로 508,37.519094,127.051901


### 격자(grid) 데이터에 추가, 독립변수 생성
* 자전거 인프라 : 있으면 1, 없으면 0
* 중,고등학교 : 최소 거리(m)
* 문화,편의시설 : 반경 100m내에 있으면 1, 없으면 0 - QGIS를 활용하여 100m 반경 내에 있는 격자 추출
    * 스포츠센터
    * 유흥업소
    * 학원
    * 슈퍼마켓, 편의점
    * 오락
    * 숙박시설
    * 문화 시설, 영화, 쇼핑
    * 지옥, 연구소, 고시원
    * 주차장

In [24]:
rental=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/4월 이전 대여소.csv',encoding='cp949',index_col=0)
rental

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,LCD(거치대수),QR(거치대수),운영방식
1,2301,현대고등학교 건너편,강남구,서울특별시 강남구 압구정로 134,37.524071,127.021790,2017-06-13,10.0,NaN,LCD
2,2302,교보타워 버스정류장(신논현역 3번출구 후면),강남구,서울특별시 강남구 봉은사로 지하 102,37.505581,127.024277,2017-06-13,10.0,NaN,LCD
3,2303,논현역 7번출구,강남구,서울특별시 강남구 학동로 지하 102,37.511517,127.021477,2017-06-13,15.0,NaN,LCD
4,2304,신영 ROYAL PALACE 앞,강남구,서울특별시 강남구 언주로 626,37.512527,127.035835,2017-06-13,10.0,NaN,LCD
5,2305,MCM 본사 직영점 앞,강남구,서울특별시 강남구 언주로 734,37.520641,127.034508,2017-06-13,10.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...
2033,1456,상아빌딩(우림시장 교차로),중랑구,서울특별시 중랑구 용마산로 494,37.595112,127.100327,2018-02-26,10.0,NaN,LCD
2034,1457,동원사거리,중랑구,서울특별시 중랑구 겸재로 229,37.589760,127.093239,2018-02-26,10.0,NaN,LCD
2035,1458,상봉터미널2,중랑구,망우동 534-58,37.597340,127.093086,2018-11-02,9.0,NaN,LCD
2036,1459,용마한신아파트사거리,중랑구,면목동 1086-79,37.580344,127.092714,2018-10-29,10.0,NaN,LCD


In [26]:
rental_place=pd.DataFrame(rental['대여소번호'])
rental_place

,대여소번호
1,2301
2,2302
3,2303
4,2304
5,2305
...,...
2033,1456
2034,1457
2035,1458
2036,1459


In [ ]:
id_list=list(bike_sisul.id)
grid_data['bike_infra']=0
grid_data.loc[(grid_data.id.apply(lambda x: x in id_list)),'bike_infra']=1

In [27]:
school_min=dis_cal(rental,school)
rental_place['school']=school_min

sports_count=is_100cal(rental,sports)
rental_place['sports']=sports_count

adult_count=is_100cal(rental,adult)
rental_place['adult']=adult_count

study_count=is_100cal(rental,study)
rental_place['study']=study_count

market_count=is_100cal(rental,market)
rental_place['market']=market_count

game_count=is_100cal(rental,game)
rental_place['game']=game_count

house_count=is_100cal(rental,house)
rental_place['house']=house_count

culture_count=is_100cal(rental,culture)
rental_place['culture']=culture_count

hell_count=is_100cal(rental,hell)
rental_place['hell']=hell_count

parking_count=is_100cal(rental,parking)
rental_place['parking']=parking_count


0
0
0
0
0
0
0
0
0
0


In [29]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking
1,2301,87.435847,0,0,1,0,0,0,0,0,0
2,2302,752.191921,0,1,1,1,1,1,1,1,1
3,2303,831.058272,0,1,1,1,1,0,1,0,0
4,2304,803.744535,0,0,0,1,0,0,0,0,0
5,2305,181.728030,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2033,1456,353.886426,0,1,1,1,1,0,0,1,0
2034,1457,515.272934,0,1,1,1,1,1,0,0,1
2035,1458,533.787400,0,1,1,1,1,1,0,0,1
2036,1459,449.461202,0,1,1,1,1,0,0,0,0


### 지하철 출구 독립변수 추가
* 지하철 출구 : 최소 거리(m)

In [31]:
subway=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/지하철 출구.csv',encoding='cp949')

In [33]:
subway.columns=['LBL_NAME','POI_NAME1','경도','위도']
subway

,LBL_NAME,POI_NAME1,경도,위도
0,3,사가정역 3번출구,127.090084,37.577807
1,4,사가정역 4번출구,127.090860,37.577723
2,8,오목교역 8번출구,126.875860,37.521749
3,3,용두역 3번출구,127.040777,37.571388
4,5,용두역 5번출구,127.039888,37.571194
...,...,...,...,...
1701,1,정릉역 1번출구,127.015583,37.600277
1702,2,정릉역 2번출구,127.015444,37.599972
1703,3,도봉산역 3번출구,127.048500,37.685667
1704,4-1,남부터미널역 4-1번출구,127.019638,37.480499


In [34]:
subway_min=dis_cal(rental,subway)
rental_place['subway']=subway_min

0


In [36]:
#rental_place.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/대여소 기준 독립변수_1117.csv',encoding='cp949',index=False)

### 공원 독립변수 추가
* 공원 : 최소 거리(m)
    * QGIS 폴리곤 전처리
    * QGIS 폴리곤 최소거리
   

In [37]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway
1,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112
2,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145
3,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936
4,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760
5,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345
...,...,...,...,...,...,...,...,...,...,...,...,...
2033,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596
2034,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819
2035,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887
2036,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998


In [38]:
park=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/공원/UPIS_SHP_ZON216(생활서비스시설-공원)/공원 최소거리.csv',encoding='cp949')

In [40]:
park['cat']=park['cat']+1

In [41]:
park

,cat,length
0,1,252.411658
1,2,317.618601
2,3,145.560776
3,4,324.674608
4,5,305.099076
...,...,...
1769,1872,146.570105
1770,1873,181.912726
1771,1874,370.992033
1772,1875,182.598290


In [54]:
park.columns=['cat','park']

In [44]:
rental_place.reset_index(drop=True,inplace=True)
rental_place['cat']=rental_place.index+1

In [56]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,cat
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,1
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,2
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,3
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,4
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,1872
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,1873
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,1874
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,1875


In [57]:
rental_place=rental_place.merge(park,on='cat',how='outer')

In [58]:
rental_place.drop('cat',axis=1,inplace=True)

In [59]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290


In [63]:
rental_place.loc[rental_place['park'].isna(),'park']=0

In [64]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290


In [65]:
#rental_place.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/대여소 기준 독립변수_1117.csv',encoding='cp949',index=False)

### 상업지역 면적 독립변수 추가
* 반경 100m 내의 상업지역 면적
    * QGIS 전처리
    * 반경 100m 버퍼 intersect 상업지역 폴리곤
    * 겹치는 폴리곤 dissolve
    * area

In [23]:
rental_place=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/대여소 기준 독립변수_1117.csv',encoding='cp949')

In [24]:
commercial=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/상업지역.csv',encoding='cp949')
commercial.columns=['대여소번호','상업지역']
commercial

,대여소번호,상업지역
0,2302,4691.636963
1,2303,3866.095459
2,2304,1988.298706
3,2305,4802.696167
4,2310,4176.172119
...,...,...
598,1448,24100.807007
599,1449,9265.285522
600,1455,12587.177246
601,1458,29576.531372


In [25]:
rental_place=rental_place.merge(commercial,on='대여소번호',how='outer')
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,NaN
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,NaN
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,NaN
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,NaN


In [26]:
rental_place.loc[rental_place['상업지역'].isna(),'상업지역']=0

In [27]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000


### 주거지역 면적 독립변수 추가
* 반경 100m 내의 상업지역 면적
    * QGIS 전처리
    * 반경 100m 버퍼 intersect 상업지역 폴리곤
    * 겹치는 폴리곤 dissolve
    * area

In [28]:
first_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/전용주거지역.csv',encoding='cp949')

In [29]:
rental_place=rental_place.merge(first_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['전용주거'].isna(),'전용주거']=0
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0


In [30]:
second_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/7층이하주거지역.csv',encoding='cp949')
second_house.head()

,대여소번호,7층이하주거지역
0,2301,16706.375122
1,2304,4165.816284
2,2305,1680.223022
3,2309,1250.753052
4,2311,2016.062744


In [31]:
rental_place=rental_place.merge(second_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['7층이하주거지역'].isna(),'7층이하주거지역']=0
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거,7층이하주거지역
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0,16706.375122
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0,0.000000
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0,0.000000
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0,4165.816284
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0,1680.223022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0,9602.471191
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0,1826.928589
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0,0.000000
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0,13109.595459


In [32]:
third_house=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/시설정보/UPIS_SHP_UQA100(주거지역)_2020/7층이상주거지역.csv')
third_house.head()

,대여소번호,고층주거
0,2301,14677.222656
1,2302,26691.501099
2,2303,27516.596802
3,2304,22835.113037
4,2305,24900.542114


In [33]:
rental_place=rental_place.merge(third_house,on='대여소번호',how='outer')
rental_place.loc[rental_place['고층주거'].isna(),'고층주거']=0
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거,7층이하주거지역,고층주거
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0,16706.375122,14677.222656
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0,0.000000,26691.501099
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0,0.000000,27516.596802
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0,4165.816284,22835.113037
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0,1680.223022,24900.542114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0,9602.471191,21781.166748
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0,1826.928589,29556.708984
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0,0.000000,1807.035522
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0,13109.595459,21792.173340


### 경사도 독립변수 추가
* 대여소 반경 100m 도로 기준
* sum((경사도*도로길이))/sum(도로길이)

In [34]:
slope=pd.read_csv('C:/Users/user/Desktop/서울시청/데이터/경사도/대여소별 경사도 가중치.csv',encoding='cp949',index_col=0)

In [35]:
slope=slope[['대여소번호','weighted_LEN_mean']]

In [36]:
slope

,대여소번호,weighted_LEN_mean
0,2301,0.845528
1,2302,2.966590
2,2303,3.572432
3,2304,4.828597
4,2305,3.447335
...,...,...
1871,1456,1.715694
1872,1457,1.477592
1873,1458,1.241584
1874,1459,1.241584


In [37]:
rental_place=rental_place.merge(slope,on='대여소번호',how='outer')
rental_place.loc[rental_place['weighted_LEN_mean'].isna(),'weighted_LEN_mean']=0
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거,7층이하주거지역,고층주거,weighted_LEN_mean
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0,16706.375122,14677.222656,0.845528
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0,0.000000,26691.501099,2.966590
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0,0.000000,27516.596802,3.572432
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0,4165.816284,22835.113037,4.828597
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0,1680.223022,24900.542114,3.447335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0,9602.471191,21781.166748,1.715694
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0,1826.928589,29556.708984,1.477592
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0,0.000000,1807.035522,1.241584
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0,13109.595459,21792.173340,1.241584


In [38]:
#rental_place.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/대여소 기준 독립변수_1119.csv',encoding='cp949',index=False)

In [39]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거,7층이하주거지역,고층주거,weighted_LEN_mean
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0,16706.375122,14677.222656,0.845528
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0,0.000000,26691.501099,2.966590
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0,0.000000,27516.596802,3.572432
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0,4165.816284,22835.113037,4.828597
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0,1680.223022,24900.542114,3.447335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0,9602.471191,21781.166748,1.715694
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0,1826.928589,29556.708984,1.477592
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0,0.000000,1807.035522,1.241584
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0,13109.595459,21792.173340,1.241584


### 도로폭 독립변수 추가
* 대여소 반경 100m 도로 기준
* sum((도로폭*도로길이))/sum(도로길이)

In [2]:
#rental_place=pd.read_csv('독립변수/대여소 기준 독립변수_1119.csv',encoding='cp949')

In [3]:
rental_place

,대여소번호,school,sports,adult,study,market,game,house,culture,hell,parking,subway,park,상업지역,전용주거,7층이하주거지역,고층주거,weighted_LEN_mean
0,2301,87.435847,0,0,1,0,0,0,0,0,0,711.946112,252.411658,0.000000,0.0,16706.375122,14677.222656,0.845528
1,2302,752.191921,0,1,1,1,1,1,1,1,1,260.995145,317.618601,4691.636963,0.0,0.000000,26691.501099,2.966590
2,2303,831.058272,0,1,1,1,1,0,1,0,0,146.357936,145.560776,3866.095459,0.0,0.000000,27516.596802,3.572432
3,2304,803.744535,0,0,0,1,0,0,0,0,0,96.046760,324.674608,1988.298706,0.0,4165.816284,22835.113037,4.828597
4,2305,181.728030,0,1,0,1,0,1,0,1,0,409.620345,305.099076,4802.696167,0.0,1680.223022,24900.542114,3.447335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1456,353.886426,0,1,1,1,1,0,0,1,0,503.544596,146.570105,0.000000,0.0,9602.471191,21781.166748,1.715694
1872,1457,515.272934,0,1,1,1,1,1,0,0,1,467.353819,181.912726,0.000000,0.0,1826.928589,29556.708984,1.477592
1873,1458,533.787400,0,1,1,1,1,1,0,0,1,39.430887,370.992033,29576.531372,0.0,0.000000,1807.035522,1.241584
1874,1459,449.461202,0,1,1,1,1,0,0,0,0,251.746998,182.598290,0.000000,0.0,13109.595459,21792.173340,1.241584


In [4]:
width=pd.read_csv('도로폭/대여소별 도로폭 가중치.csv',encoding='cp949',index_col=0)

In [6]:
width=width[['대여소번호','mean_width']]

In [11]:
rental_place=rental_place.merge(width,on='대여소번호')

In [13]:
#rental_place.to_csv('C:/Users/user/Desktop/서울시청/데이터/독립변수/대여소 기준 독립변수_1130.csv',encoding='cp949',index=False)